# Lab Assignment Four: Multi-Layer Perceptron

### Authors
- Juliana Antonio
- Xiaona Hang
- Chuanqi Deng


## 1. Load, Split, and Balance 


References:
1. https://www.kaggle.com/muonneutrino/us-census-demographic-data/data